<a href="https://colab.research.google.com/github/HindJamal97/Algorithm-for-Massive-Data/blob/main/A_priori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#FIRST PART. 
#Install kaggle and upload + unzip our dataset. 

In [2]:
!pip install -q kaggle

In [3]:
#After installing kaggle, we need to upload the kaggle.json file with the API token given from out Kaggle account. 

In [4]:
from google.colab import files
files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hindjamal","key":"5b2794cdd1f8c072feba270d5a765fa6"}'}

In [5]:
#Now we need to create a Kaggle folder, copy the kaggle.json to the folder created and finally we need to grant an approppriate permission for the json file to act. 

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
#To list all the available datasets we have in Kaggle we use the following command

In [8]:
! kaggle datasets list

ref                                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                                 Data Science Salaries 2023 💸                         25KB  2023-04-13 09:55:16          29685        822  1.0              
tawfikelmetwally/automobile-dataset                                   Car information dataset                               6KB  2023-05-28 18:26:48           2841         78  0.9411765        
fatihb/coffee-quality-data-cqi                                        Coffee Quality Data (CQI May-2023)                   22KB  2023-05-12 13:06:39           5203        107  1.0              
mohithsairamreddy/salary-data 

In [9]:
#Finally, for our aim, we need to upload our dataset, and given that the url is the following one, we can download it as follows: 

In [10]:
!kaggle datasets download -d  xhlulu/medal-emnlp

100% 6.81G/6.82G [01:17<00:00, 110MB/s]
100% 6.82G/6.82G [01:17<00:00, 94.1MB/s]


In [11]:
from zipfile import ZipFile


In [12]:
file_name = "medal-emnlp.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Done")

Done


In [13]:
#SECOND PART. 
#Install Spark and construct a software that can be used to find frequent itemset (RDD). 

In [14]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.3.2
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [15]:
!pip install py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
!java -version

openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [17]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [18]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [19]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=70faca69b7db81e0385c54317c29aad7f9c4206b273052e0450a6ddb2e5f97bc
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [20]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext   

In [21]:
conf = SparkConf()
conf = (conf
        .setAppName("AMD-MeDal")
        .setMaster('local[*]')
        .set('spark.executor.memory', '8G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '0'))
sc = SparkContext(conf=conf)

In [22]:
spark = SQLContext(sc)

/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [23]:
sc

<SparkContext master=local[*] appName=AMD-MeDal>

In [24]:
spark

In [25]:
#Upload the file

In [26]:
dataset_path = '/content/full_data.csv'

In [27]:
df  = spark.read.format('csv') \
                .option('header', True) \
                .option('multiline', True) \
                .option('quote', '"') \
                .option('escape', '"') \
                .load(dataset_path)
df.printSchema()

root
 |-- TEXT: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LABEL: string (nullable = true)



In [28]:
df.show()

+--------------------+--------------------+--------------------+
|                TEXT|            LOCATION|               LABEL|
+--------------------+--------------------+--------------------+
|alphabisabolol ha...|                  56|           substrate|
|a report is given...|24|49|68|113|137|172|carcinosarcoma|re...|
|the virostatic co...|                  55|           substrate|
|rmi rmi and rmi a...|   25|82|127|182|222|compounds|compoun...|
|a doubleblind stu...|22|26|28|77|90|14...|oxazepam|placebo|...|
|stroma from eithe...|         6|82|84|107|red cells|serum|a...|
|the effect of the...|                4|13|major|pentose pho...|
|in one experiment...|        32|44|76|135|feeding|feeding|a...|
|the presence of a...|7|15|63|137|199|2...|active|study|acti...|
|the reaction of g...|     113|203|209|250|stable|assay|bind...|
|choline acetyltra...|                  44|             caudate|
|increasing concen...|                  81|        displacement|
|the properties of...|   

In [29]:
df.head(3)

[Row(TEXT='alphabisabolol has a primary antipeptic action depending on dosage which is not caused by an alteration of the phvalue the proteolytic activity of pepsin is reduced by percent through addition of bisabolol in the ratio of the antipeptic action of bisabolol only occurs in case of direct contact in case of a previous contact with the ATP the inhibiting effect is lost', LOCATION='56', LABEL='substrate'),
 Row(TEXT='a report is given on the recent discovery of outstanding immunological properties in ba ncyanoethyleneurea having a low molecular mass m experiments in ds CS bearing wistar rats have shown that ba at a dosage of only about percent ld mg kg and negligible lethality percent results in a REC rate of percent without hyperglycemia and in one test of percent with hyperglycemia under otherwise unchanged conditions the REF substance ifosfamide if a further development of cyclophosphamide applied without hyperglycemia in its most efficient dosage of percent ld mg kg brought a

In [30]:
# Get row count
rows = df.count()

# Get columns count
cols = len(df.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")
     

DataFrame Dimensions : (14393619, 3)
DataFrame Rows count : 14393619
DataFrame Columns count : 3


In [31]:
DF = df.select('TEXT')

In [32]:
DF.show()

+--------------------+
|                TEXT|
+--------------------+
|alphabisabolol ha...|
|a report is given...|
|the virostatic co...|
|rmi rmi and rmi a...|
|a doubleblind stu...|
|stroma from eithe...|
|the effect of the...|
|in one experiment...|
|the presence of a...|
|the reaction of g...|
|choline acetyltra...|
|increasing concen...|
|the properties of...|
|primary amines re...|
|a purification pr...|
|dihydrofolate red...|
|ionization effect...|
|kinetic analyses ...|
|the nearultraviol...|
|the circular pola...|
+--------------------+
only showing top 20 rows



In [33]:
#Pre-processing of the data that we have!!

In [34]:
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 kB 10.8 MB/s eta 0:00:00


In [35]:
import sparknlp
import math
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline
import pyspark.sql.types as T
from typing import List

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import csv
import time
from tqdm import tqdm

In [37]:
from pyspark.sql.functions import lower, col, udf

In [38]:
DF_lowercase = DF.withColumn("processed_text", lower(col("text")))

In [39]:
DF_lowercase.show()

+--------------------+--------------------+
|                TEXT|      processed_text|
+--------------------+--------------------+
|alphabisabolol ha...|alphabisabolol ha...|
|a report is given...|a report is given...|
|the virostatic co...|the virostatic co...|
|rmi rmi and rmi a...|rmi rmi and rmi a...|
|a doubleblind stu...|a doubleblind stu...|
|stroma from eithe...|stroma from eithe...|
|the effect of the...|the effect of the...|
|in one experiment...|in one experiment...|
|the presence of a...|the presence of a...|
|the reaction of g...|the reaction of g...|
|choline acetyltra...|choline acetyltra...|
|increasing concen...|increasing concen...|
|the properties of...|the properties of...|
|primary amines re...|primary amines re...|
|a purification pr...|a purification pr...|
|dihydrofolate red...|dihydrofolate red...|
|ionization effect...|ionization effect...|
|kinetic analyses ...|kinetic analyses ...|
|the nearultraviol...|the nearultraviol...|
|the circular pola...|the circul

In [40]:
from pyspark.sql.functions import regexp_replace, trim, col, lower


In [41]:
def removePunctuation(column):
   return lower(trim(regexp_replace(column,'\p{Punct}',''))).alias('sentence')

In [42]:
RemoveP = DF_lowercase.select(removePunctuation(col('processed_text')).alias("processed_text"))
RemoveP.show(5)

+--------------------+
|      processed_text|
+--------------------+
|alphabisabolol ha...|
|a report is given...|
|the virostatic co...|
|rmi rmi and rmi a...|
|a doubleblind stu...|
+--------------------+
only showing top 5 rows



In [43]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import trim,split,explode,col

In [44]:
tokenizer = Tokenizer(inputCol='processed_text', outputCol='words_token')
Data = RemoveP.withColumn("processed_text", trim(col("processed_text")))
DATA = tokenizer.transform(Data)
DATA.show(20)

+--------------------+--------------------+
|      processed_text|         words_token|
+--------------------+--------------------+
|alphabisabolol ha...|[alphabisabolol, ...|
|a report is given...|[a, report, is, g...|
|the virostatic co...|[the, virostatic,...|
|rmi rmi and rmi a...|[rmi, rmi, and, r...|
|a doubleblind stu...|[a, doubleblind, ...|
|stroma from eithe...|[stroma, from, ei...|
|the effect of the...|[the, effect, of,...|
|in one experiment...|[in, one, experim...|
|the presence of a...|[the, presence, o...|
|the reaction of g...|[the, reaction, o...|
|choline acetyltra...|[choline, acetylt...|
|increasing concen...|[increasing, conc...|
|the properties of...|[the, properties,...|
|primary amines re...|[primary, amines,...|
|a purification pr...|[a, purification,...|
|dihydrofolate red...|[dihydrofolate, r...|
|ionization effect...|[ionization, effe...|
|kinetic analyses ...|[kinetic, analyse...|
|the nearultraviol...|[the, nearultravi...|
|the circular pola...|[the, circ

In [45]:
from pyspark.ml.feature import StopWordsRemover

In [46]:
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
Df = remover.transform(DATA).select("processed_text",'words_clean')
Df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [47]:
#The following code is trying to eliminate some duplicated words that we have in each sentence, but we still do not know if this is the best possile action. 

In [48]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType,StringType    

In [49]:
get_uniques=udf(lambda x: list(set(x)), ArrayType(StringType()))

DataBase = Df.withColumn("words_clean", get_uniques(Df.words_clean))
DataBase.show(10)

+--------------------+--------------------+
|      processed_text|         words_clean|
+--------------------+--------------------+
|alphabisabolol ha...|[case, alteration...|
|a report is given...|[pronounced, deve...|
|the virostatic co...|[substance, prese...|
|rmi rmi and rmi a...|[hypotensive, lev...|
|a doubleblind stu...|[clear, obtained,...|
|stroma from eithe...|[lysis, pathway, ...|
|the effect of the...|[pathway, concent...|
|in one experiment...|[feeding, rumens,...|
|the presence of a...|[vmax, bindingdep...|
|the reaction of g...|[step, stable, ad...|
+--------------------+--------------------+
only showing top 10 rows



In [50]:
from pyspark.sql.functions import collect_set
from pyspark.sql.types import IntegerType

In [51]:
slen = udf(lambda s: len(s), IntegerType())
DB = DataBase.withColumn("word_count", slen(DataBase.words_clean))

In [52]:
DB.show()

+--------------------+--------------------+----------+
|      processed_text|         words_clean|word_count|
+--------------------+--------------------+----------+
|alphabisabolol ha...|[case, alteration...|        26|
|a report is given...|[pronounced, deve...|       123|
|the virostatic co...|[substance, prese...|        33|
|rmi rmi and rmi a...|[hypotensive, lev...|       105|
|a doubleblind stu...|[clear, obtained,...|        93|
|stroma from eithe...|[lysis, pathway, ...|        60|
|the effect of the...|[pathway, concent...|        25|
|in one experiment...|[feeding, rumens,...|        60|
|the presence of a...|[vmax, bindingdep...|        96|
|the reaction of g...|[step, stable, ad...|        81|
|choline acetyltra...|[using, ionexchan...|        76|
|increasing concen...|[protein, globula...|        65|
|the properties of...|[protein, builtin...|        95|
|primary amines re...|[protein, react, ...|        60|
|a purification pr...|[step, dhf, equil...|        91|
|dihydrofo

In [53]:
import pyspark.sql.functions as F

In [54]:
DB.createOrReplaceTempView("df_view")

baskets = spark.sql("SELECT processed_text, words_clean AS Basket, word_count FROM df_view")

In [55]:
baskets.show()

+--------------------+--------------------+----------+
|      processed_text|              Basket|word_count|
+--------------------+--------------------+----------+
|alphabisabolol ha...|[case, alteration...|        26|
|a report is given...|[pronounced, deve...|       123|
|the virostatic co...|[substance, prese...|        33|
|rmi rmi and rmi a...|[hypotensive, lev...|       105|
|a doubleblind stu...|[clear, obtained,...|        93|
|stroma from eithe...|[lysis, pathway, ...|        60|
|the effect of the...|[pathway, concent...|        25|
|in one experiment...|[feeding, rumens,...|        60|
|the presence of a...|[vmax, bindingdep...|        96|
|the reaction of g...|[step, stable, ad...|        81|
|choline acetyltra...|[using, ionexchan...|        76|
|increasing concen...|[protein, globula...|        65|
|the properties of...|[protein, builtin...|        95|
|primary amines re...|[protein, react, ...|        60|
|a purification pr...|[step, dhf, equil...|        91|
|dihydrofo

In [56]:
baskets.printSchema()

root
 |-- processed_text: string (nullable = true)
 |-- Basket: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word_count: integer (nullable = true)



In [57]:
# We reduce the dimension of the dataframe for the naive approach
sample = baskets.limit(2000)

In [58]:
sample.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [59]:
# Get row count
rows = sample.count()

# Get columns count
cols = len(sample.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")

DataFrame Dimensions : (2000, 3)
DataFrame Rows count : 2000
DataFrame Columns count : 3


In [60]:
# We will convert our preprocessed dataframe into an rdd
# with the following structure (key, value)

input_rdd = sample.rdd.map(lambda x: (1, list(set(x[1]))))

# This is our input rdd
input_rdd.take(1)

[(1,
  ['case',
   'alteration',
   'direct',
   'alphabisabolol',
   'effect',
   'contact',
   'bisabolol',
   'depending',
   'primary',
   'previous',
   'action',
   'ratio',
   'atp',
   'caused',
   'antipeptic',
   'pepsin',
   'dosage',
   'phvalue',
   'proteolytic',
   'addition',
   'percent',
   'reduced',
   'inhibiting',
   'occurs',
   'lost',
   'activity'])]

In [64]:
THRESHOLD_PERCENTAGE = 0.1
n_of_baskets = input_rdd.countByKey()[1]

THRESHOLD = math.ceil(n_of_baskets * THRESHOLD_PERCENTAGE)

In [65]:
print(THRESHOLD)

200


In [66]:
# Extract all singletons
singleton_itemsets_rdd = input_rdd \
                          .flatMap(lambda x: x[1]) \
                          .map(lambda x: (x,1))

singleton_itemsets_w_frequencies_rdd = singleton_itemsets_rdd \
                                        .reduceByKey(lambda x,y: x + y)

# Filter singletons with a frequency higher than the THRESHOLD
frequent_singleton_itemsets_rdd = singleton_itemsets_w_frequencies_rdd \
                                    .filter(lambda x: x[1] > THRESHOLD)

frequent_singleton_itemsets_rdd.take(10)

[('effect', 439),
 ('activity', 587),
 ('obtained', 261),
 ('t3', 217),
 ('determined', 204),
 ('blood', 234),
 ('one', 301),
 ('well', 207),
 ('conditions', 257),
 ('different', 263)]

In [67]:
# We can then compute frequent pairs 
# Thanks to the monotonicity property we won't generate false negatives

# Extract all frequent singleton without frequency
frequent_singleton_itemsets_wout_freq_rdd = frequent_singleton_itemsets_rdd \
                                              .map(lambda x: (1, x[0]))

# Generate all candidate pairs 
candidate_pairs_rdd = frequent_singleton_itemsets_wout_freq_rdd \
                      .join(frequent_singleton_itemsets_wout_freq_rdd) \
                      .filter(lambda x: len(set(x[1])) == len(x[1])) \
                      .map(lambda x: (tuple(sorted(x[1])), 1)) \
                      .reduceByKey(lambda x,y: x)

# Generate a list of candidate pairs 
candidate_pairs_list = candidate_pairs_rdd.map(lambda x: x[0]).collect()
broadcasted_candidate_pairs_list = sc.broadcast(candidate_pairs_list)

# Compute all possible pairs on our whole dataset
input_w_unique_key_rdd = input_rdd \
                        .map(lambda x: x[1]) \
                        .zipWithUniqueId() \
                        .flatMap(lambda x: [(x[1], word) for word in x[0]])

pair_itemsets_rdd = input_w_unique_key_rdd \
                  .join(input_w_unique_key_rdd)

# Filter only pairs that are in the candidate frequent pairs list and compute their frequency
frequent_pairs_itemsets_rdd = pair_itemsets_rdd \
                                .filter(lambda x: x[1] in broadcasted_candidate_pairs_list.value) \
                                .map(lambda x: (x[1],1)) \
                                .reduceByKey(lambda x,y: x + y) \
                                .filter(lambda x: x[1] > THRESHOLD)

frequent_pairs_itemsets_rdd.take(5)

[(('acid', 'ph'), 215),
 (('enzyme', 'ph'), 209),
 (('activity', 'enzyme'), 251),
 (('activity', 'ph'), 271),
 (('c', 'ph'), 202)]

In [68]:
frequent_pairs_itemsets_wout_freq_rdd = frequent_pairs_itemsets_rdd.map(lambda x: (1, x[0]))

# Generate all candidate triplets 
candidate_triplets_rdd = frequent_pairs_itemsets_wout_freq_rdd \
                      .join(frequent_pairs_itemsets_wout_freq_rdd) \
                      .map(lambda x: (1, sum(x[1], ()))) \
                      .map(lambda x: (1, tuple(set(x[1]))) ) \
                      .filter(lambda x: len(x[1]) == 3) \
                      .map(lambda x: (tuple(sorted(x[1])), 1)) \
                      .reduceByKey(lambda x,y: x)

# Generate a list of candidate triplets 
candidate_triplets_list = candidate_triplets_rdd.map(lambda x: x[0]).collect()
broadcasted_candidate_triplets_list= sc.broadcast(candidate_triplets_list)

# Compute all triplets on our whole dataset
triplets_itemsets_rdd = input_w_unique_key_rdd \
                        .map(lambda x: (x[0], (x[1], ))) \
                        .join(pair_itemsets_rdd)  \
                        .map(lambda x: (1, sum(x[1], ())))

# Filter only triplets that are in the candidate frequent triples list and compute their frequency
frequent_triplets_itemsets_rdd = triplets_itemsets_rdd \
                                .filter(lambda x: x[1] in broadcasted_candidate_triplets_list.value) \
                                .map(lambda x: (x[1],1)) \
                                .reduceByKey(lambda x,y: x + y) \
                                .filter(lambda x: x[1] > THRESHOLD)

frequent_triplets_itemsets_rdd.take(5)

[]